In [3]:
from pathlib import Path
# from dabble_bot import *
import dabble_bot as dbb
import torch
from importlib import reload

path = Path.cwd()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


cpu


In [ ]:
model = torch.load(path / 'model_back_ups/dabble_bot_1.2')

In [7]:
model.estimate_loss(100)

{'train': tensor(1.6328), 'val': tensor(2.5296)}

In [2]:
model = dbb.DabbleBot(dbb.wsample)
model.to(device)
model.load_state_dict(torch.load(path / 'model_back_ups/dabble_bot_weights_1.2', weights_only=True))
model.estimate_loss(100)

{'train': tensor(1.6495), 'val': tensor(2.5116)}

In [4]:
import gensim.downloader
glove = gensim.downloader.load('glove-wiki-gigaword-50')

import numpy as np
from sklearn.cluster import KMeans

word_space = None
for word in dbb.wsample:
  try:
    embedded_word = glove[word]
    if word_space == None:
      word_space = torch.tensor(embedded_word)
    else:
      word_space = torch.cat((word_space, torch.tensor(embedded_word)))
  except:
    pass

word_space = word_space.reshape(-1, 50)
word_space.shape

[==================================================] 100.0% 66.0/66.0MB downloaded


torch.Size([85282, 50])

In [6]:
km = KMeans(n_clusters=50, random_state=42).fit(word_space);
print(km.inertia_)

1198657.6594824456


In [7]:
ws = word_space.numpy().tolist()
c_dict = {c:[] for c in set(km.labels_.tolist())}
for c, w in zip(km.labels_, ws):
  c_dict[c].append(w)

In [8]:
c_dict[1]

[[-0.7243899703025818,
  -0.5722299814224243,
  -0.2380799949169159,
  -0.48276999592781067,
  0.6955500245094299,
  -0.8849800229072571,
  0.4614099860191345,
  -0.6348299980163574,
  0.17674000561237335,
  1.9017000198364258,
  0.797950029373169,
  0.24368999898433685,
  -0.15036000311374664,
  0.6281599998474121,
  0.02249700017273426,
  -0.35069000720977783,
  1.2690999507904053,
  0.23694999516010284,
  0.3019599914550781,
  -0.06718800216913223,
  -0.7234200239181519,
  -0.8370400071144104,
  -0.08215700089931488,
  0.18900999426841736,
  -1.3056999444961548,
  0.5475900173187256,
  1.780500054359436,
  0.5869899988174438,
  0.9708600044250488,
  -0.45274001359939575,
  -0.25176000595092773,
  -0.4719499945640564,
  -0.43046000599861145,
  0.12231999635696411,
  0.776669979095459,
  0.2935500144958496,
  0.31356000900268555,
  0.5516200065612793,
  -0.5414800047874451,
  0.20529000461101532,
  0.5212500095367432,
  -0.8334199786186218,
  0.5341899991035461,
  0.4546799957752228,


In [10]:
km.cluster_centers_[0]

array([-0.28590768, -0.26424367, -0.8050763 , -0.29632469, -0.43644735,
        0.31568273,  0.4361361 ,  0.28317818, -0.51761675,  0.96668932,
        0.14618466,  0.21068266,  0.32505064,  0.57217597, -0.49752058,
       -0.52031961, -0.23723251,  0.14187871,  0.15897331, -0.39148372,
       -0.03741285,  0.15395141, -0.13924913,  0.12016201,  0.42807575,
        0.54625651, -0.47302067,  0.12321227, -0.13273549, -0.52274449,
        0.43747263, -0.23062708,  0.53005733,  0.58279341,  0.1929717 ,
        0.17227988,  0.39310128, -0.26027734, -0.10954905,  0.31976856,
        0.17831046,  0.02657738, -0.41287858, -0.02518811,  0.13849137,
        0.17099219,  0.56687736, -0.06034458, -0.67061533, -0.02349556])

In [12]:
glove.similar_by_vector(km.cluster_centers_[0])

[('accompaniment', 0.8511331677436829),
 ('solos', 0.8449597954750061),
 ('improvisations', 0.8340027928352356),
 ('bluesy', 0.824953556060791),
 ('choruses', 0.8222054243087769),
 ('riff', 0.8221226930618286),
 ('stylings', 0.8204526305198669),
 ('flute', 0.8203752040863037),
 ('riffs', 0.8166735768318176),
 ('percussive', 0.8118392825126648)]

In [13]:
glove.most_similar(km.cluster_centers_[0])

[('accompaniment', 0.8511331677436829),
 ('solos', 0.8449597954750061),
 ('improvisations', 0.8340027928352356),
 ('bluesy', 0.824953556060791),
 ('choruses', 0.8222054243087769),
 ('riff', 0.8221226930618286),
 ('stylings', 0.8204526305198669),
 ('flute', 0.8203752040863037),
 ('riffs', 0.8166735768318176),
 ('percussive', 0.8118392825126648)]